In [1]:
print("Hello world")

Hello world


In [2]:
%pip install google-adk --quiet
%pip install litellm --quiet
%pip install python-dotenv --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
litellm 1.80.10 requires grpcio<1.68.0,>=1.62.3; python_version < "3.14", but you have grpcio 1.76.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.76.0 requires grpcio>=1.76.0, but you have grpcio 1.67.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv() 

True

In [4]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts
from typing import Optional, Dict, Any

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [5]:
# Define Model Constants for easier use 
MODEL_GPT = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, 
                                messages=[{"role": "user", 
                                           "content": "Are you ready?"}], 
                                tools=[]))

print("\nOpenAI is ready for use.")

ModelResponse(id='chatcmpl-CrctI8aTkwRx7D6S0Qc4b1zPfKXMR', created=1766896916, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_deacdd5f6f', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready! How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None, image_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

OpenAI is ready for use.


In [6]:
# ## How to setup Neo4j connection using environment variables
# docker run -d \
#   --name neo4j \
#   -p 7474:7474 -p 7687:7687 \
#   -e NEO4J_AUTH=neo4j/password123 \
#   neo4j:latest

In [7]:
# if you are coming back after a session restart, start the container again
# docker start neo4j or docker restart neo4j.

In [8]:
## add this to yor .env file
# NEO4J_URI=bolt://127.0.0.1:7687
# NEO4J_USERNAME=neo4j
# NEO4J_PASSWORD=password123
# NEO4J_DATABASE=neo4j

In [9]:
%pip install neo4j-for-adk --quiet


ERROR: Could not find a version that satisfies the requirement neo4j-for-adk (from versions: none)
ERROR: No matching distribution found for neo4j-for-adk
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install neo4j --quiet

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb

In [12]:
import neo4j
print(neo4j.__version__)

6.0.3


In [ ]:
# Comprehensive Neo4j Diagnostics
import os
import socket
import time
from neo4j import GraphDatabase

print("=== Neo4j Connection Diagnostics ===\n")

# 1. Check environment variables
print("1. Environment Variables:")
print(f"   NEO4J_URI: {os.getenv('NEO4J_URI', 'Not set')}")
print(f"   NEO4J_USERNAME: {os.getenv('NEO4J_USERNAME', 'Not set')}")
print(f"   NEO4J_PASSWORD: {'*' * len(os.getenv('NEO4J_PASSWORD', '')) if os.getenv('NEO4J_PASSWORD') else 'Not set'}")
print(f"   NEO4J_DATABASE: {os.getenv('NEO4J_DATABASE', 'Not set')}\n")

# 2. Check if ports are open
print("2. Port Connectivity Test:")
def check_port(host, port):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((host, port))
        sock.close()
        return result == 0
    except:
        return False

ports_to_check = [
    ('127.0.0.1', 7687, 'Neo4j Bolt (IPv4)'),
    ('localhost', 7687, 'Neo4j Bolt (localhost)'),
    ('127.0.0.1', 7474, 'Neo4j Browser (IPv4)'),
    ('localhost', 7474, 'Neo4j Browser (localhost)')
]

for host, port, desc in ports_to_check:
    is_open = check_port(host, port)
    status = "✅ OPEN" if is_open else "❌ CLOSED"
    print(f"   {desc} ({host}:{port}): {status}")

print()

# 3. Try different connection approaches
print("3. Connection Attempts:")

# Try with explicit IPv4
try:
    driver = GraphDatabase.driver("bolt://127.0.0.1:7687", auth=("neo4j", "password123"))
    with driver.session() as session:
        result = session.run("RETURN 'IPv4 Direct' as test, timestamp() as time")
        record = result.single()
        print(f"   ✅ IPv4 Direct: {record['test']} at {record['time']}")
    driver.close()
except Exception as e:
    print(f"   ❌ IPv4 Direct: {type(e).__name__}: {str(e)[:100]}...")

# Try with localhost
try:
    driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password123"))
    with driver.session() as session:
        result = session.run("RETURN 'localhost' as test, timestamp() as time")
        record = result.single()
        print(f"   ✅ localhost: {record['test']} at {record['time']}")
    driver.close()
except Exception as e:
    print(f"   ❌ localhost: {type(e).__name__}: {str(e)[:100]}...")

# Try with environment variables
try:
    uri = os.getenv('NEO4J_URI', 'bolt://127.0.0.1:7687')
    username = os.getenv('NEO4J_USERNAME', 'neo4j')
    password = os.getenv('NEO4J_PASSWORD', 'password123')
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        result = session.run("RETURN 'Env Vars' as test, timestamp() as time")
        record = result.single()
        print(f"   ✅ Environment Vars: {record['test']} at {record['time']}")
    driver.close()
except Exception as e:
    print(f"   ❌ Environment Vars: {type(e).__name__}: {str(e)[:100]}...")

print("\n=== Recommendations ===")
print("If all connection attempts fail:")
print("1. Wait 30-60 seconds after container start for Neo4j to fully initialize")
print("2. Check Docker logs: docker logs neo4j")
print("3. Verify container is running: docker ps | grep neo4j")
print("4. Try recreating container if issues persist:")
print("   docker rm -f neo4j")
print("   docker run -d --name neo4j -p 7474:7474 -p 7687:7687 -e NEO4J_AUTH=neo4j/password123 neo4j:latest")

=== Neo4j Connection Diagnostics ===

1. Environment Variables:
   NEO4J_URI: bolt://127.0.0.1:7687
   NEO4J_USERNAME: neo4j
   NEO4J_PASSWORD: ***********
   NEO4J_DATABASE: neo4j

2. Port Connectivity Test:
   Neo4j Bolt (IPv4) (127.0.0.1:7687): ✅ OPEN
   Neo4j Bolt (localhost) (localhost:7687): ✅ OPEN
   Neo4j Browser (IPv4) (127.0.0.1:7474): ✅ OPEN
   Neo4j Browser (localhost) (localhost:7474): ✅ OPEN

3. Connection Attempts:
   ✅ IPv4 Direct: IPv4 Direct at 1766896951004
   ✅ localhost: localhost at 1766896951240
   ✅ Environment Vars: Env Vars at 1766896951305

=== Recommendations ===
If all connection attempts fail:
1. Wait 30-60 seconds after container start for Neo4j to fully initialize
2. Check Docker logs: docker logs neo4j
3. Verify container is running: docker ps | grep neo4j
4. Try recreating container if issues persist:
   docker rm -f neo4j
   docker run -d --name neo4j -p 7474:7474 -p 7687:7687 -e NEO4J_AUTH=neo4j/password123 neo4j:latest


In [ ]:
# Diagnostic: Test connecting to Neo4j using 127.0.0.1 to avoid IPv6 issues
import os
from neo4j import GraphDatabase

print('NEO4J_URI (env):', os.getenv('NEO4J_URI'))
uri = os.getenv('NEO4J_URI') or 'bolt://127.0.0.1:7687'
username = os.getenv('NEO4J_USERNAME') or 'neo4j'
password = os.getenv('NEO4J_PASSWORD') or 'password123'

try:
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as s:
        r = s.run("RETURN 'ok' AS ok")
        print('Connection test result:', r.single()['ok'])
    driver.close()
except Exception as e:
    import traceback
    print('Connection failed:', type(e).__name__, e)
    traceback.print_exc()


NEO4J_URI (env): bolt://127.0.0.1:7687
Connection test result: ok


In [14]:
# Define a basic tool -- send a parameterized cypher query
def say_hello(person_name: str) -> dict:
    """Formats a welcome message to a named person. 

    Args:
        person_name (str): the name of the person saying hello

    Returns:
        dict: A dictionary containing the results of the query.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'query_result' key with an array of result rows.
              If 'error', includes an 'error_message' key.
    """
    return graphdb.send_query("RETURN 'Hello to you, ' + $person_name AS reply",
    {
        "person_name": person_name
    })


In [15]:
# Example tool usage (optional test)
print(say_hello("Harsh"))

{'status': 'error', 'error_message': "Couldn't connect to 127.0.0.1:7687 (resolved to ('127.0.0.1:7687',)):\nFailed to read four byte Bolt handshake response from server ResolvedIPv4Address(('127.0.0.1', 7687)) (deadline Deadline(timeout=60.0))"}


**Optional Reading**

- An `Agent` in Google ADK orchestrates the interaction between the user, the LLM, and the available tools

- you configure it with several key parameters:
    * `name`: A unique identifier for this agent (e.g., "friendly_cypher_agent\_v1").  
    * `model`: Specifies which LLM to use. you'll use the `llm` variable we defined above.  
    * `description`: A summary of the agent's overall purpose. This is like public documentation that helps other agents decide when to delegate tasks to *this* agent.  
    * `instruction`: Detailed guidance given to the LLM on how this agent should behave, its persona, goals, and specifically *how and when* to utilize its assigned `tools`.  
    * `tools`: A list containing the actual Python tool functions the agent is allowed to use (e.g., `[say_hello]`).

**Best Practice:** 
- Provide clear and specific `instruction` prompts. The more detailed the instructions, the better the LLM can understand its role and how to use its tools effectively. Be explicit about error handling if needed.
- Choose descriptive `name` and `description` values. These are used internally by ADK and are vital for features like automatic delegation (covered later).

In [16]:
# Define the Cypher Agent
hello_agent = Agent(
    name="hello_agent_v1",
    model=llm, # defined earlier in a variable
    description="Has friendly chats with a user.",
    instruction="""You are a helpful assistant, chatting with a user. 
                Be polite and friendly, introducing yourself and asking who the user is. 

                If the user provides their name, use the 'say_hello' tool to get a custom greeting.
                If the tool returns an error, inform the user politely. 
                If the tool is successful, present the reply.
                """,
    tools=[say_hello], # Pass the function directly
)

print(f"Agent '{hello_agent.name}' created.")

Agent 'hello_agent_v1' created.


Let's assume we have a single user talking to the agent in a single session. Let's create this user, the session and the runner:
* `SessionService`: Responsible for managing conversation history and state for different users and sessions. The `InMemorySessionService` is a simple implementation that stores everything in memory, suitable for testing and simple applications. It keeps track of the messages exchanged.  
* `Runner`: The engine that orchestrates the interaction flow. It takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the `SessionService`, and yields events representing the progress of the interaction.

In [17]:
app_name = hello_agent.name + "_app"
user_id = hello_agent.name + "_user"
session_id = hello_agent.name + "_session_01"
    
# Initialize a session service and a session
session_service = InMemorySessionService()
await session_service.create_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id
)
    
runner = Runner(
    agent=hello_agent,
    app_name=app_name,
    session_service=session_service
)

Here's what's happening:
 
1. Package the user query into the ADK `Content` format.
2. Call`runner.run_async` (providing it with user/session context and the new message)
4. Iterate through the **Events** yielded by the runner. Events represent steps in the agent's execution (e.g., tool call requested, tool result received, intermediate LLM thought, final response).  
5. Identify and print the **final response** event using `event.is_final_response()`.

**Why `async`?** Interactions with LLMs and potentially tools (like external APIs) are I/O-bound operations. Using `asyncio` allows the program to handle these operations efficiently without blocking execution.

In [18]:
user_message = "Hello, I'm Harsh, nice to meet you!"
print(f"\n>>> User Message: {user_message}")

# Prepare the user's message in ADK format
content = types.Content(role='user', parts=[types.Part(text=user_message)])

final_response_text = "Agent did not produce a final response." # Default will be replaced if the agent produces a final response.


# We iterate through events to find the final answer.
verbose = True
async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
    if verbose:
        print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")
    
    # Key Concept: is_final_response() marks the concluding message for the turn.
    if event.is_final_response():
        if event.content and event.content.parts:
            final_response_text = event.content.parts[0].text # Assuming text response in the first part
        elif event.actions and event.actions.escalate: # Handle potential errors/escalations
            final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
        break # Stop processing events once the final response is found

print(f"<<< Agent Response: {final_response_text}")


>>> User Message: Hello, I'm Harsh, nice to meet you!
  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'person_name': 'Harsh'
    },
    id='call_jZGbF47hNqHEBv4rALLxYYXA',
    name='say_hello'
  )
)] role='model'
  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_jZGbF47hNqHEBv4rALLxYYXA',
    name='say_hello',
    response={
      'error_message': """Couldn't connect to 127.0.0.1:7687 (resolved to ('127.0.0.1:7687',)):
Failed to read four byte Bolt handshake response from server ResolvedIPv4Address(('127.0.0.1', 7687)) (deadline Deadline(timeout=60.0))""",
      'status': 'error'
    }
  )
)] role='user'
  [Event] Author: hello_agent_v1, Type: Event, Final: True, Content: parts=[Part(
  text='Hello Harsh, nice to meet you too! It seems like there was a technical issue when trying to generate a custom greeting for you. I a

In [19]:
class AgentCaller:
    """A simple wrapper class for interacting with an ADK agent."""
    
    def __init__(self, agent: Agent, runner: Runner, 
                 user_id: str, session_id: str):
        """Initialize the AgentCaller with required components."""
        self.agent = agent
        self.runner = runner
        self.user_id = user_id
        self.session_id = session_id


    def get_session(self):
        return self.runner.session_service.get_session(app_name=self.runner.app_name, user_id=self.user_id, session_id=self.session_id)

    
    async def call(self, user_message: str, verbose: bool = False):
        """Call the agent with a query and return the response."""
        print(f"\n>>> User Message: {user_message}")

        # Prepare the user's message in ADK format
        content = types.Content(role='user', parts=[types.Part(text=user_message)])

        final_response_text = "Agent did not produce a final response." 
        
        # Key Concept: run_async executes the agent logic and yields Events.
        # We iterate through events to find the final answer.
        async for event in self.runner.run_async(user_id=self.user_id, session_id=self.session_id, new_message=content):
            # You can uncomment the line below to see *all* events during execution
            if verbose:
                print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

            # Key Concept: is_final_response() marks the concluding message for the turn.
            if event.is_final_response():
                if event.content and event.content.parts:
                    # Assuming text response in the first part
                    final_response_text = event.content.parts[0].text
                elif event.actions and event.actions.escalate: # Handle potential errors/escalations
                    final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
                break # Stop processing events once the final response is found

        print(f"<<< Agent Response: {final_response_text}")
        return final_response_text


In [20]:
async def make_agent_caller(agent: Agent, initial_state: Optional[Dict[str, Any]] = {}) -> AgentCaller:
    """Create and return an AgentCaller instance for the given agent."""
    app_name = agent.name + "_app"
    user_id = agent.name + "_user"
    session_id = agent.name + "_session_01"
    
    # Initialize a session service and a session
    session_service = InMemorySessionService()
    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
        state=initial_state
    )
    
    runner = Runner(
        agent=agent,
        app_name=app_name,
        session_service=session_service
    )
    
    return AgentCaller(agent, runner, user_id, session_id)


In [21]:
hello_agent_caller = await make_agent_caller(hello_agent)

# We need an async function to await our interaction helper
async def run_conversation():
    await hello_agent_caller.call("Hello I'm Harsh")

    await hello_agent_caller.call("I am excited")

# Execute the conversation using await
await run_conversation()


>>> User Message: Hello I'm Harsh
<<< Agent Response: Hello to you, Harsh! It's nice to meet you. How can I assist you today?

>>> User Message: I am excited
<<< Agent Response: That's wonderful to hear, Harsh! What are you excited about?
